In [1]:
import pandas as pd
from tqdm.auto import tqdm
base = "https://synapseaisolutionsa.blob.core.windows.net/public/AdventureWorks"

# load list of tables
df_tables = pd.read_csv(f"{base}/adventureworks.csv", names=["table"])

for table in (pbar := tqdm(df_tables['table'].values)):
    pbar.set_description(f"Uploading {table} to lakehouse")

    # download
    df = pd.read_parquet(f"{base}/{table}.parquet")

    # save as lakehouse table
    spark.createDataFrame(df).write.mode('overwrite').saveAsTable(table)

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 3, Finished, Available, Finished)

  0%|          | 0/28 [00:00<?, ?it/s]

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 4, Finished, Available, Finished)

In [6]:
import pandas as pd
from tqdm.auto import tqdm
base = "https://synapseaisolutionsa.blob.core.windows.net/public/AdventureWorks"

# load list of tables
df_tables = pd.read_csv(f"{base}/adventureworks.csv", names=["table"])

for table in (pbar := tqdm(df_tables['table'].values)):
    pbar.set_description(f"Uploading {table} to lakehouse")

    # download
    df = pd.read_parquet(f"{base}/{table}.parquet")

    # save as lakehouse table
    spark.createDataFrame(df).write.mode('overwrite')\
          .option("overwriteSchema", "true")\
          .save(f"abfss://fabric_aiskills_demo@onelake.dfs.fabric.microsoft.com/AdvWorksLH.Lakehouse/Files/Data/{table}")

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 11, Finished, Available, Finished)

  0%|          | 0/28 [00:00<?, ?it/s]

StatementMeta(, 31e6d638-ffde-4a62-9ae5-d5cb4901a8cb, 12, Finished, Available, Finished)